# A Simple Random Forest Model to Identify Putative Genes Using Conserved Cysteine Residues
##### Name: Marvellous O. Oyebanjo
##### UB Number: 24013601
##### Word Count: 1967

# INTRODUCTION

<div align="justify">Elucidating how functional variants occur and persist in populations is an interesting topic in evolutionary biology. These variants are important for the persistence of polymorphism. How said variants exist and are maintained depends on the affecting selection pressures. Balancing selection (BS) is requisite for the perseverance of the genetic diversity of phenotypes in populations via overdominant and frequency-dependent selection [1,2]. Most importantly, the co-occurrence of varying adaptive variants of a trait is linked to BS [1]. BS is characterised by its transient background signatures in genomes, forming the base of the analytical framework used to identify BS [3]. Genomic regions undergoing strong balancing selection, especially those involving short, hypervariable protein-coding genes, are difficult to identify and or annotate because the signature is subtle and short-lived.</div>
<br>
<div align="justify">Although genomic regions undergoing BS have hypervariable gene sequences, there are yet conserved residues within these regions. These residues are usually under positive selection, allowing for the existence of different alleles [4]. One of such residues is cysteine. Cysteine residues (CR) are vital for protein function via disulfide bridge formation and protein-to-protein intermolecular interactions [5]. Cysteine’s abundance or the effect of its mutation is tied to its function; so, even if Cysteine is not as abundant as other amino acids, the slightest mutation could have undesirable phenotypic effects, provided the CRs are functionally active [6]. BS helps maintain cysteine conservation in plant populations by stopping the proliferation of deleterious mutations that would interrupt cysteine functions.</div>
<br>
<div align="justify">The genomic region analysed in this study contains a short, exonic gene, with less than 150 amino acids, characterised by 8 CRs, which may be emblematic of the gene of interest. Traditional annotation tools for identifying putative genes using conserved residues in regions undergoing BS are riddled with false positives and low detection power [7]. This limitation is due to the BS signatures being microscopic and momentary. Hence, other selection pressures can overshadow BS signatures or cause them to be mistaken for each other. However, machine learning (ML) can detect putative genes in regions undergoing positive selection or low-signature BS [7]. <div>
<br>
<div align="justify">DNA/Protein sequences undergoing BS have numerous features with nonlinear interactions, making it complex for simple regression frameworks. A ML model of notable interest in putative gene prediction is Random Forest (RF) – a tree-based ensemble statistical method – which is well-suited to handle the problematic nature of genomic features’ high dimensionality [8]. Furthermore, RF is useful for supervised pattern matching in sequence data because: One, it is natively built to generate a trained decision tree that performs binary classification, which, in this case, is <i>does this sequence contain CRs or not?</i> Two, RF has a high predictive ability over other ML classification models, such as support vector machines, K-nearest-neighbours, and neural networks [9]. Consequently, these features make RF a choice tool for recognising patterns in genomic regions under strong BS. <div>
<br>
<div align="justify">This study was aimed at evaluating the efficacy of ML, specifically, Random Forest, in identifying putative gene regions, mainly characterised by 8 conserved CRs, within a genomic region undergoing strong BS.</div>

<br>

# METHODS
## Data Information
<div align="justify">The sequence data used for this analysis contained 11 FASTA files, which represented a genome region of about 20kbp, undergoing strong balancing selection.<div>
<br>
    
## Analytical Pipeline
### Installation of Packages
The following Python packages were installed using the pip install option (`pip install biopython scikit-learn pandas`):
- `biopython` (Bio): The package was necessary for processing sequence data in Python. Specifically, it was used to read and parse sequences via `SeqIO`, perform BLAST operations via `Bio.Blast.NCBIWW`, and extract methods and structures associated to sequences via `Seq` and `SeqRecord`. 
- `scikit-learn` (sklearn): Seeing as this is a machine learning-based analysis, the sklearn package was necessary. This package was used to build a Random Forest classifier to categorise sequences based on their features. 
- `pandas`: This package was used for data manipulation. Specifically, it was used to convert the standard outputs of the machine learning predictions, BLAST analysis, and sequence data into data frames and tables. 

In [2]:
pip install biopython scikit-learn pandas

Note: you may need to restart the kernel to use updated packages.


<br>

### Imported Python Modules
The following modules were used for this analysis:
- `os`: This module was requisite for interacting with directories and reading the 11 FASTA files. 
- `re`: This module was used for string pattern matching, specifically, it was used for extracting parts of the sequence data, such as headers. 
- `pd` (pandas): This module, pandas, was imported for organising and manipulating data into tables.
- `From Bio import SeqIO, Seq`: The SeqIO module was imported for accessing FASTA files and sequences iteration, while the Seq module was used for sequence manipulation, such as translating DNA to protein, complementing, and reversing.
- `From IPython.display import display, HTML`: These modules were used to render custom-formatted HTML tables.
- `From sklearn.ensemble import RandomForestClassifier`: This module was used for categorising sequences using ensemble learning. 
- `From sklearn.model_selection import train_test_split`: This module was needed for splitting the sequence features into training and testing subgroups for model generalisation.
- `From sklearn.metrics import classification_report`: This module was used to generate a summary report, containing metrics of the model performance, such as accuracy, precision, F1-score, and recall.
- `From Bio.Blast import NCBIWWW, NCBIXML`: The NCBIWW module was used to query the NCBI BLAST server for homologous protein sequences in the NCBI non-redundant (nr) database, while the NCBIXML was used to obtain results, such as query coverage, percent identity, E-value, and alignment length in XML format.
- `From Bio.Seq import Seq`: Same as function as `from Bio.Seq import Seq`.
- `From Bio.SeqRecord import SeqRecord`: This SeqRecord module serves as a data container for the Seq object and metadata, such as sequence annotations, description and ID. Additionally, this module helps provide the required format for `qblast()` queries. 
<br>

### Python Functions
#### Sequence Padding
<div align="justify">A helper function called <code>pad_seq</code> was created to ensure that the nucleotide sequences were multiples of 3 before translation, generating uniformly structured sequences with standardised lengths. This will avoid errors when using the translate option from Biopython. Additionally, this method achieves translation without trimming; thus, preventing the loss of biologically meaningful data.<div>
<br>
    
#### Open Reading Frame Extraction
<div align="justify">The <code>extract_orfs</code> function was used to identify open reading frames (ORFs) or exons, which are continuous regions with start and stop codons. The function searches all 6 ORFs: 3 forward and reverse, each. The <code>min_len</code> was set to 30 ORFs because very short amino acids (AA) or sequences do not often translate into functional proteins. The <code>max_len</code> was capped at 450 due to excessively long ORFs possibly extending into non-coding regions, leading to pseudo-AA sequences. Consequently, <code>max_len-min_len</code> range provides a balance between specificity and sensitivity, enabling the detection of biologically meaningful peptides without an increase in irrelevance.<div>
<br>
       
#### Extracting Sequence Features and Labelling Criterion
<div align="justify">The <code>extract_features</code> function was essential for the conversion of AA/peptide sequences into numerical data for ML classification. The sequences are strings and cannot be accepted by the ML model. The relaxed labelling criterion, 7 to 9 cysteine count (CC), was chosen to exclude short peptides with low CC and excessively long CC. Furthermore, many cysteine-rich peptides (CRPs) have 4-12 CC [10], and the (genomic) region of interest is characterised by 8 CC. Hence, the 7 to 9 CC is a fair delineation for the identification of functionally relevant number of cysteines. Peptides with 7-9 CC are labelled as positive (1) and, those without, negative (0). <code>Length</code>, <code>cysteine_count</code>, and <code>label</code> are the extracted features and form the training dataset.<div>
<br>
        
#### Create ML Input Data from FASTA Files
<div align="justify">The <code>create_dataset_from_folder</code> function pre-processes the input data by converting raw sequences (FASTA files) into data suitable for ML classification. The function creates a pre-processing pipeline that: One, performs batch processing of multiple input files; Two, uses <code>SeqIO.parse()</code> to iterate through FASTA files, <code>extract_orfs()</code> function to perform candidate peptide prioritisation (ORFs identification) on DNA sequences, and <code>extract_features()</code> to convert peptide sequences into ML-readable numerical input. The output is a pandas dataframe, consisting of binary label, sequence length, CC, and peptide sequence – with each row representing an ORF. <div>
<br>
    
#### ML Model Training
<div align="justify">The <code>train_classifier</code> function was written to differentiate between CRPs and cysteine-poor peptides (CPPs). This function uses the length and CC features to train the ML model. Stratified splitting between training and testing datasets is initially attempted to ensure class distribution. However, if there are fewer subclasses than expected, to prevent a runtime error, random splitting is employed. The <code>RandomForestClassifier</code> was used because it works well with high-dimensional data. <code>test_size=0.3</code> was used because it is the commonly used data splitting ratio [11]; the training set is 0.7. <code>random_state=42</code> was set as the fixed seed to ensure reproducibility. <code>n_estimators</code> was set to 100, giving it a good baseline for training time and performance. <code>class_weight=balanced</code> was specified to handle class imbalance and reduce bias.<div>
<br>
    
#### Predictions from ML Model 
<div align="justify">The <code>predict_on_file</code> function used a trained ML model to identify CRPs. <code>.predict_proba()</code> assigns a confidence score for each ORF. Only high-confidence predictions (probability > 0.9) are returned, ensuring only highly likely gene candidates are selected.<div>
<br>
    
### BLAST Validation
<div align="justify">Using the high-scoring predictions, a BLAST validation was performed to determine if the predicted peptides matched experimentally-validated CRPs. Additionally, BLAST hits usually have annotations, which could help prescribe plausible biological functions of the ML predictions or identify homologs in other species.<div> 


<br>

# RESULTS AND DISCUSSION
All the functions were integrated into a pipeline. Below are the results. 

In [27]:
# Import modules
import os
import re
import pandas as pd
from Bio import SeqIO, Seq
from IPython.display import display, HTML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ---------- Pad Sequences to get Multiples of 3 ----------
def pad_seq(seq):
    """
    Pads a nucleotide sequence so its length is a multiple of 3.

    Sequence padding ensures the sequence can be translated into protein
    without frameshift issues. If the sequence length, when divided by 3, has remainders,
    'N' characters (which can be any nucleotide) are added to the end.

    Parameters:
    ----------
    seq (str): 
        The nucleotide sequence (e.g., "AGGCAGUGG") to be padded.

    Returns:
    -------
    Bio.Seq.Seq (str): 
        A padded sequence object from Biopython, with length a multiple of 3.
    """
    # First, calculate the remainder when the sequence length is divided by 3
    remainder = len(seq) % 3
    # If the sequence length is not a multiple of 3 (triplets), pad the sequence with 'N' to address the problem
    if remainder != 0:
        seq += 'N' * (3 - remainder)
    # Returns a Biopython object     
    return Seq.Seq(seq)

# ---------- ORF Extraction ----------
def extract_orfs(seq, min_len=30, max_len=450):
    """
    Extract open reading frames (ORFs) from all six reading frames of a DNA sequence.

    An ORF is defined here as a region of a translated protein sequence between 
    start and stop codons, however, within a specified length range.

    --------
    Parameters:
    ----------
    seq (str): Bio.Seq.Seq
        A Biopython `Seq` object representing the nucleotide sequence.
    
    min_len (int): Optional (default=30)
        Minimum allowed amino acid (AA/aa) length of an ORF to be considered valid. 30 was chosen to allow for an extensive search.

    max_len (int): Optional (default=450)
        Maximum allowed amino acid length of an ORF to be considered valid.

    --------
    Returns:
    -------
    List[str]
        A list of valid ORF peptide sequences (as strings) from all six frames.
    """
    # Start with an empty list
    peptides = []
    # Create 6 open reading frames: 3 forward and 3 reverse complement
    frames = [seq[i:] for i in range(3)] + [seq.reverse_complement()[i:] for i in range(3)]
    # For Loop to iterate through ORFs
    for frame in frames:
        # Pad the frame using 'pad_seq' function so its length is a multiple of 3
        padded = pad_seq(str(frame))
        # Translate the entire padded sequence into protein sequences
        prot = padded.translate(to_stop=False)
        # Starting position 
        start = 0
        # Use in-built RegEx ('re' module) to find stop codons (*) within protein sequences/ORFs
        for match in re.finditer(r'\*', str(prot)):
            # The end of the current ORF
            stop = match.start()
            # Retain ORFs that meet the 'min_len' and 'max_len' criteria
            if min_len <= (stop - start) <= max_len:
                peptides.append(str(prot[start:stop]))
            # Resets the starting position for the next potential ORF to the character just after the stop codon
            start = match.end()
    return peptides

# ---------- Feature Extraction with Relaxed Label ----------
def extract_features(peptide):
    """
    Extract features from a peptide sequence for machine learning (ML) classification.

    The 'extract_features' function calculates the peptide length, the number of cysteine (C) residues,
    and assigns a binary label using a simple criterion: sequences with 7 to 9 cysteines 
    (inclusive) are labelled as positive samples (label = 1), while others are negative (label = 0).

    -------
    Parameters:
    -------
    peptide (str): 
        The amino acid sequence of the peptide.
    
    -------
    Returns:
    -------
    dict: 
        A dictionary containing:
            - 'length': Total number of amino acids in the sequence.
            - 'cysteine_count': Number of 'C' (cysteine) residues.
            - 'sequence': The original peptide sequence.
            - 'label': 1 if 7 <= cysteine_count <= 9, else 0.
    """
    # Calculate the number of cysteine residues
    c_count = peptide.count('C')
    # Returns relevant metric
    return {
        "length": len(peptide), # Peptide length
        "cysteine_count": c_count, # Number of Cysteine residues
        "sequence": peptide, # Peptide sequence
        "label": 1 if 7 <= c_count <= 9 else 0  # Labelling criterion
    }

# ---------- Load Dataset from .fas Folder ----------
def create_dataset_from_folder(folder):
    """
    Extracts peptide features from all FASTA files (.fas) in a specified folder.

    This function scans a directory for FASTA files, extracts ORFs from each sequence 
    using the `extract_orfs` function, computes features (like length, cysteine count, and labels) 
    for each ORF via the `extract_features` function, and compiles all feature dictionaries into a single pandas DataFrame.

    --------
    Parameters:
    -------
    folder (str): 
        Path to the directory containing `.fas` files.

    -------
    Returns:
    -------
    pd.DataFrame: 
        A DataFrame where each row represents a peptide ORF with computed features.
    """
    # Create an empty list to hold feature dictionaries from 'extract_orfs' output
    all_data = []
    # For loop to iterate through files in the folder
    for filename in os.listdir(folder):
        # If statement to allow for the processing of files ending with '.fas', '.fasta', or '.fa'; '.lower' renders it case-insensitive
        if filename.lower().endswith((".fas", ".fasta", ".fa")):
            # Full file path
            path = os.path.join(folder, filename)
            # For loop to parse FASTA files and extract ORFs for each sequence
            for record in SeqIO.parse(path, "fasta"):
                # Collects ORFs for each sequence
                orfs = extract_orfs(record.seq)
                # For loop to iterate through ORFs and append them to the 'all_data' object
                for orf in orfs:
                    all_data.append(extract_features(orf))
    # Returns a dataframe containing the feature dictionaries                
    return pd.DataFrame(all_data)

# ---------- Train Model ----------
def train_classifier(df):
    """
    Train a Random Forest classifier to identify cysteine-rich peptides.

    --------
    Parameters:
    -----------
    df: 
    pandas.DataFrame:
        A DataFrame containing peptide features including 'length', 'cysteine_count', and a binary 'label'
        indicating whether the peptide is considered cysteine-rich (1) or not (0).

    --------
    Returns:
    --------
    clf: 
    RandomForestClassifier:
        A trained Random Forest classifier.
    
    --------
    Notes:
    ------
    - If the dataset has insufficient positive/negative samples for stratified splitting,
      it falls back to random splitting to avoid errors.
    - Prints a classification report to evaluate model performance on the test set.
    """
    # Show label/class distribution as a styled table: how many samples exist in 1 (cysteine-rich) and 0 (cysteine-poor) classes
    label_counts = df["label"].value_counts().rename_axis("Label").reset_index(name="Count")
    label_title = "<h2 style='font-family:monospace;'>Table 1: Label Distribution</h2>"
    styled_labels = (
        label_counts.style
        .hide(axis="index")
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-family', 'monospace')]},
            {'selector': 'td', 'props': [('text-align', 'left'), ('font-family', 'monospace')]},
        ])
        .set_properties(**{
            'text-align': 'left',
            'font-family': 'monospace',
        })
    )
    display(HTML(label_title))
    display(styled_labels) 
    
    # Statement to check IF Stratified Splitting is possible; a minimum of 2 samples is needed from each Label
    if df["label"].value_counts().min() < 2:
        print("⚠️ Not enough positive examples for stratified splitting. Using random split.")
        # Use Random Splitting if Stratified Splitting is not possible
        stratify = None
    else:
        # Stratify to maintain class balance in test/training data split
        stratify = df["label"]
    # Specify features
    X = df[["length", "cysteine_count"]]
    # Specify label (0s or 1s)
    y = df["label"]
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=stratify, test_size=0.3, random_state=42)
    # Initialise the Random Forest Classifier with class balancing
    clf = RandomForestClassifier(
        n_estimators=100, # Number of trees in the forest
        class_weight="balanced", # Handles class imbalancing
        random_state=42 # For debugging and reproducibility
    )
    # Train the model based on the training data
    clf.fit(X_train, y_train)
    
    # Print the evaluation metric for model performance on the test set
    report_str = classification_report(y_test, clf.predict(X_test))

    # Format and display as styled HTML
    report_html = f"""
    <h2 style='font-family:monospace;'>Table 2: Classification Report</h2>
    <pre style='font-family:monospace'>{report_str}</pre>
    """
    display(HTML(report_html))
    
    # Returns the trained classifier
    return clf

# ---------- Run ML model on New File ----------
def predict_on_file(model, fas_file):
    """
    Run predictions on a FASTA file using a trained classification model.

    This function parses a given .fas/.fasta/.fa file, extracts ORFs (open reading frames),
    computes features for each ORF, and predicts whether the ORF is a cysteine-rich peptide
    using a trained classifier. Only predictions with high confidence (probability > 0.9)
    are returned.

    --------
    Parameters:
    --------
    model (sklearn classifier):
        A trained classifier with a '.predict_proba()' method.
    fas_file (str): 
        Path to the input FASTA file.

    --------
    Returns:
    --------
    list of tuples: 
        Each tuple contains (sequence, prediction score, cysteine positions).
    """
    # Create an empty list
    predictions = []
    # For loop to parse input FASTA files
    for record in SeqIO.parse(fas_file, "fasta"):
        # Extract ORFs
        orfs = extract_orfs(record.seq)
        # For loop to iterate through ORFS
        for orf in orfs:
            # Extract features for model prediction
            features = extract_features(orf)
            # Create a dataframe to hold the features for prediction
            X_pred = pd.DataFrame([[features["length"], features["cysteine_count"]]],
                                  columns=["length", "cysteine_count"])
            # Probability prediction that this ORF is a positive (1; cysteine-rich) or negative (0; cysteine-poor) 
            prob = model.predict_proba(X_pred)[0][1]
            # Retain probability > 0.9 (high-confidence)
            if prob > 0.9:
                # Get positions of cysteine residues in the sequence
                cysteine_positions = [i for i, aa in enumerate(orf) if aa == 'C']
                # Append the ORF sequence, probability, and cysteine positions in the 'predictions' object
                predictions.append((orf, prob, cysteine_positions))
    # Return the sorted prediction by score in descending order
    return sorted(predictions, key=lambda x: -x[1])

# ---------- Running the pipeline ----------
if __name__ == "__main__":
    print("🧬 Processing .fas files in folder...")
    df = create_dataset_from_folder("Downloads/fasta_files") # specify filepath
    print(f"✅ Found {len(df)} ORFs.")

    print("🤖 Training model...")
    model = train_classifier(df)

# --- Collect results from predictions ---
results = [] # Create an empty list

test_folder = "Downloads/fasta_files" # Specify filepath to get file names

for filename in os.listdir(test_folder):
    if filename.lower().endswith(".fas"):
        file_path = os.path.join(test_folder, filename)
        hits = predict_on_file(model, file_path)
        for seq, score, positions in hits:
            results.append({
                "File": filename,
                "Score": round(score, 3),
                "Sequence Length (aa)": len(seq),
                "Cysteine Count": seq.count('C'),
                "Cysteine Positions": str(positions),  # convert list to string for table
                "Full Sequence": seq
            })

# --- Convert results to DataFrame ---
df = pd.DataFrame(results)

# Sort the DataFrame by the 'Score' column in descending order
df_sorted = df.sort_values(by="Score", ascending=False)

# Define table title
title_html = "<h2 style='font-family:monospace;'>Table 3: Top Predicted Cysteine-Rich ORFs</h2>"

# Style the DataFrame
styled_df = (
    df_sorted.style
    .hide(axis="index")  # hides the row indices
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'left'), ('font-family', 'monospace')]},
        {'selector': 'td', 'props': [('text-align', 'left'), ('font-family', 'monospace'), ('white-space', 'pre-wrap')]},
    ])
    .set_properties(**{
        'text-align': 'left',
        'font-family': 'monospace',
        'white-space': 'pre-wrap',
        'max-width': '300px',
    })
    .format(precision=3)
    .background_gradient(subset=["Score"], cmap="Greens", gmap=df_sorted["Score"].rank(ascending=True))
)

# Display title and custom-styled table
display(HTML(title_html))
display(styled_df)

🧬 Processing .fas files in folder...
✅ Found 4071 ORFs.
🤖 Training model...


Label,Count
0,4060
1,11


File,Score,Sequence Length (aa),Cysteine Count,Cysteine Positions,Full Sequence
Cgr-B-H7-9M04_corrected_assembled_S-locus.fas,0.990,65,8,"[11, 12, 21, 36, 46, 50, 57, 64]",YTMILLIYIDKCCYMFIYIYICVYHDVLTFLYYKSICHDTWYAITICYDICLIYHDICLIYHDIC
Cgr-B-H2-1H03_corrected_assembled_S-locus.fas,0.980,63,7,"[0, 1, 3, 5, 8, 25, 33]",CCSCFCFFCMLQRIVSYRRKPFIRPCSKNGNEICKSVKYYYVNRMYLSTEVPFFKKVHRKHKI
Cgr-B-H3-7N24_corrected_assembled_S-locus.fas,0.980,63,7,"[0, 1, 3, 5, 8, 25, 33]",CCSCFCFFCMLQRIVSYRRKPFIRPCSKNGNEICKSVKYYYVNRMYLSTEVPFFKKVHRKHKI
Cgr-B-H6-5I19_corrected_assembled_S-locus.fas,0.950,62,7,"[12, 16, 26, 35, 44, 46, 61]",PSNGVSAHYPTICINGCKSIKRKHILCVYIIFLLRCVHNITQYLCVCILIVFIRKHYTTFLC
Cgr-B-H2-12I07_corrected_assembled_S-locus.fas,0.930,46,7,"[20, 23, 28, 31, 37, 40, 44]",NHDYNYIYHYHYISKDSLKSCIVCNVLFCIMCFYIKHCFLCYQLCV
Cgr-B-H3-15B12_corrected_assembled_S-locus.fas,0.930,46,7,"[20, 23, 28, 31, 37, 40, 44]",NHDYNYIYHYHYISKDSLKSCIVCNVLFCIMCFYIKHCFLCYQLCV
Cgr-B-H4-1G22_corrected_assembled_S-locus.fas,0.930,46,7,"[20, 23, 28, 31, 37, 40, 44]",NHDYNYIYHYHYISKDSLKSCIVCNVLFCIMCFYIKHCFLCYQLCV
Cgr-B-H5-19I12_corrected_assembled_S-locus.fas,0.930,46,7,"[20, 23, 28, 31, 37, 40, 44]",NHDYNYIYHYHYISKDSLKSCIVCNVLFCIMCFYIKHCFLCYQLCV
Cgr-B-H6-3P02_corrected_trim_S-locus.fas,0.920,89,9,"[24, 40, 50, 58, 72, 74, 83, 85, 88]",ESSSRVDLSIVYLEIYYRKNVLRNCGLSEQTEVEAWTRDKCDISDNFIGKCGDDGDRECVADFYRIHVTVTRCGCREFFKSRICNCKIC
Cgr-B-H7-9M04_corrected_assembled_S-locus.fas,0.910,228,7,"[6, 19, 42, 69, 75, 122, 178]",VVISIKCDQSFTSLNAFVHCFGVDLIGSAYLSEHLDGDSGIPCIEDIVSLVLTVFPAKGLVPSFSLGVFCVVDRLCDGLVFTKPDIYVKQFHPVLLQSQKVEAYQVDMASSFPASWMAKSIGCVLGWFIKDPFKATIQQEFFLRPFVSFFLVAEVLALKAAISIVLALGVSKLTYYSECKEVNLLFIVDGQANEVDGIFADIKFQRNKFLTFSFLVLHITVVFVTSPP


<br>
<div align="justify">Using the FASTA files, the ML model found 4071 ORFs (Table 1). 4060 of the ORFs were cysteine-poor, while 11 had a CC between 7 to 9, making them CRPs per the model specification. Only 9 of the 11 predicted ORFs had prediction scores > 0.9. The 2 low-confidence predictions were Cgr-B-H4-6I24 and Cgr-B-H5-14J22. Cgr-B-H7-9M04 had two high-confidence predictions, 0.99 and 0.91. The former had an AA length and CC of 65 and 8, respectively, while the latter, 228 and 7. The high AA length of the latter prediction casts doubts on it, as the CRP is reportedly within an exonic region of less than 150 AAs. Notably, the 0.99 prediction for Cgr-B-H7-9M04 had a CC of 8, the exact number of cysteine residues associated with the putative gene, existing across positions 11-12-21-36-46-50-57-64.<div>
<br>

In [19]:
# BLAST Validation
# Import modules
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
from IPython.display import display, HTML

# Choose predicted sequences from ML for BLAST
top_preds = results[:10]  # The ML made 10 predictions, so, the predictions will be validated using BLASTP

# Create an empty list to collect dictionaries for BLAST hits
blast_data = []

# For loop to iterate through each prediction and make a BLASTp query for each
for i, result in enumerate(top_preds):
    # Retrieve full peptide sequences from 'result' or use sequence preview if unavailable
    seq = result.get("Full Sequence", "") or result.get("Sequence Preview", "")
    # Prediction score from ML classification
    score = result.get("Score", 0.0)
    filename = result.get("File", f"Unknown_{i+1}")

    # If empty sequences are present, skip them
    if not seq:
        print(f"⚠️ Sequence missing for index {i}, skipping.")
        continue

    print(f"🔍 Running BLASTp for sequence {i+1} from {filename} (Score: {score:.3f})")

    # Perform data wrapping on peptide sequences using 'SeqRecord' to convert it to FASTA format for 'qblast'
    record = SeqRecord(Seq(seq), id=f"predicted_{i+1}", description=f"{filename} | Score_{score:.3f}")

    # Run BLASTp
    try:
        result_handle = NCBIWWW.qblast("blastp", "nr", record.format("fasta"))
        # parse XML to object
        blast_record = NCBIXML.read(result_handle) 

        # For loop to parse through the top 3 hits
        for alignment in blast_record.alignments[:3]:
            # For loop to examine only the first HSP
            for hsp in alignment.hsps[:1]:  
                # Percentage of query covered
                query_coverage = round((hsp.align_length / len(seq)) * 100, 2)
                # Percentage of identical residues in aligned regions
                percent_identity = round((hsp.identities / hsp.align_length) * 100, 2)
                # Append rows to 'blast_data' list
                blast_data.append({
                    "Filename": filename,
                    "Sequence ID": record.id,
                    "Score": float(score),
                    "Hit Description": alignment.hit_def,
                    "Query Coverage (%)": query_coverage,
                    "Percent Identity (%)": percent_identity,
                    "E-value": hsp.expect,
                    "Alignment Length": hsp.align_length
                })

    except Exception as e:
        print(f"❌ BLAST failed for {record.id}: {e}")

# Convert to DataFrame
blast_df = pd.DataFrame(blast_data)

# Sort safely by score 
if "Score" in blast_df.columns:
    blast_df = blast_df.sort_values(by="Score", ascending=False)


# Display with custom styling
title_blast = "<h2 style='font-family:monospace;'>Table 4: BLAST Results</h2>"

# Styling
styled_blast = (
    blast_df.style
    .hide(axis="index")  # hides the row indices
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'left'), ('font-family', 'monospace')]},
        {'selector': 'td', 'props': [('text-align', 'left'), ('font-family', 'monospace'), ('white-space', 'pre-wrap')]},
    ])
    .set_properties(**{
        'text-align': 'left',
        'font-family': 'monospace',
        'white-space': 'pre-wrap',
        'max-width': '300px',
    })
    .format(precision=3)
)

# Display custom-style table and title
display(HTML(title_blast))
display(styled_blast)

🔍 Running BLASTp for sequence 1 from Cgr-B-H2-12I07_corrected_assembled_S-locus.fas (Score: 0.930)
🔍 Running BLASTp for sequence 2 from Cgr-B-H2-1H03_corrected_assembled_S-locus.fas (Score: 0.980)
🔍 Running BLASTp for sequence 3 from Cgr-B-H3-15B12_corrected_assembled_S-locus.fas (Score: 0.930)
🔍 Running BLASTp for sequence 4 from Cgr-B-H3-7N24_corrected_assembled_S-locus.fas (Score: 0.980)
🔍 Running BLASTp for sequence 5 from Cgr-B-H4-1G22_corrected_assembled_S-locus.fas (Score: 0.930)
🔍 Running BLASTp for sequence 6 from Cgr-B-H5-19I12_corrected_assembled_S-locus.fas (Score: 0.930)
🔍 Running BLASTp for sequence 7 from Cgr-B-H6-3P02_corrected_trim_S-locus.fas (Score: 0.920)
🔍 Running BLASTp for sequence 8 from Cgr-B-H6-5I19_corrected_assembled_S-locus.fas (Score: 0.950)
🔍 Running BLASTp for sequence 9 from Cgr-B-H7-9M04_corrected_assembled_S-locus.fas (Score: 0.990)
🔍 Running BLASTp for sequence 10 from Cgr-B-H7-9M04_corrected_assembled_S-locus.fas (Score: 0.910)


Filename,Sequence ID,Score,Hit Description,Query Coverage (%),Percent Identity (%),E-value,Alignment Length
Cgr-B-H7-9M04_corrected_assembled_S-locus.fas,predicted_9,0.990,uncharacterized protein [Panulirus ornatus],61.540,35.000,0.569,40
Cgr-B-H6-5I19_corrected_assembled_S-locus.fas,predicted_8,0.950,glutathione S-transferase kappa 1 isoform X1 [Rhea pennata],62.900,35.900,1.803,39
Cgr-B-H6-3P02_corrected_trim_S-locus.fas,predicted_7,0.920,SCRp [Arabidopsis lyrata] >gb|KAG7542035.1| hypothetical protein ISN45_Aa07g020611 [Arabidopsis thaliana x Arabidopsis arenosa] >gb|WWS21479.1| SCR01 [Arabidopsis halleri],77.530,73.910,0.000,69
Cgr-B-H6-3P02_corrected_trim_S-locus.fas,predicted_7,0.920,SCR60 [Arabidopsis halleri],70.790,39.680,0.000,63
Cgr-B-H6-3P02_corrected_trim_S-locus.fas,predicted_7,0.920,SCR02 [Arabidopsis halleri],88.760,31.650,0.001,79
Cgr-B-H7-9M04_corrected_assembled_S-locus.fas,predicted_10,0.910,"unnamed protein product, partial [Arabidopsis halleri]",88.600,52.480,0.000,202
Cgr-B-H7-9M04_corrected_assembled_S-locus.fas,predicted_10,0.910,Ribonuclease H domain [Arabidopsis thaliana x Arabidopsis arenosa],88.600,53.470,0.000,202
Cgr-B-H7-9M04_corrected_assembled_S-locus.fas,predicted_10,0.910,hypothetical protein ARALYDRAFT_908334 [Arabidopsis lyrata subsp. lyrata] >emb|CAH8270024.1| unnamed protein product [Arabidopsis lyrata],88.600,51.490,0.000,202


<br>
<div align="justify">The prediction <i>Cgr-B-H6-3P02</i> was interesting because, even though it had a score of 0.92, the AA length (89), CC (9), and cysteine positions (24-40-50-58-72-74-83-85-88) made it promising. A high-scoring prediction by the Random Forest algorithm [9], although beneficial, is not the only deciding factor for the biological significance of the peptide prediction or the presence of the putative gene. The cysteine spacing (a prerequisite for disulfide bonding) and BLAST similarity report are equally important. For instance, <i>Cgr-B-H7-9M04</i>, albeit with a high-scoring prediction, had an adjacent cysteine pair (11-12), which would impede disulfide bond formation between the CRs [12]. Furthermore, BLAST analysis revealed a low-scoring percentage identity (Table 4) and an E-value to an uncharacterised protein.<div>
<br>
    
<div align="justify">However, <i>Cgr-B-H6-3P02</i>, with close clusters of CRs and a 0.92 prediction score, is somewhat spaced apart, allowing for a plausibly stable disulfide bond formation [12]. Additionally, BLAST (Table 4) revealed a 73.91% identity and 0.0 E-value similarity between <i>Cgr-B-H6-3P02</i> and Scarecrow Protein (SCRp), an actual S-locus CRP gene in <i>Arabidopsis spp.</i> [13]. Consequently, this predicted CRP is likely to be biologically functional, and the genomic region represented by <i>Cgr-B-H6-3P02</i> could contain the candidate gene.<div> 
<br>

<div align="justify">Surprisingly, no high-scoring similarities were found for <i>Cgr-B-H2-12I07</i>, <i>Cgr-B-H2-1H03</i>, <i>Cgr-B-H3-15B12</i>, <i>Cgr-B-H3-7N24</i>, <i>Cgr-B-H4-1G22</i>, and <i>Cgr-B-H5-19I12</i>, even though they all had at least 0.90 prediction scores (Table 3). This discrepancy can be elucidated by the classification report (Table 2). For the cysteine-poor peptides (0), the ML predicted 1219 in the testing set, all with 1.00 Precision, Recall, and F1-Score, suggesting a perfect prediction. However, for CRPs (1), only 3 predictions in the test data were perfect. This small number indicates that the CRPs are underrepresented, and, although the model performed well on this test data, it may not do so with other datasets. This underrepresentation bias can be corrected by augmenting the datasets with simulated CRPs or additional data [14]. Moreover, the Synthetic Minority Over-sampling (SMOTE) can be used to correct the class imbalance.<div>
<br>

<div align="justify">Alternatively, a motif-based search of ORFs using regular expressions can be used to identify CRPs linked to regions undergoing balancing selection. Similarly, structural prediction and disulfide bond formation using AlphaFold [15] and DISULFIND [16] are non-ML approaches that can identify CRPs associated with genes under strong balancing selection.<div>

# REFERENCES
[1]	V. Llaurens, A. Whibley, and M. Joron, “Genetic architecture and balancing selection: the life and death of differentiated variants,” Molecular Ecology, vol. 26, no. 9, pp. 2430–2448, Mar. 2017, doi: 10.1111/mec.14051.
<br>

[2]	A. M. Andres et al., “Targets of Balancing Selection in the Human Genome,” Molecular Biology and Evolution, vol. 26, no. 12, pp. 2755–2764, Aug. 2009, doi: 10.1093/molbev/msp190.
<br>

[3]	A. Fijarczyk and W. Babik, “Detecting balancing selection in genomes: limits and prospects,” Molecular Ecology, vol. 24, no. 14, pp. 3529–3545, Jun. 2015, doi: 10.1111/mec.13226.
<br>

[4]	D. Koenig et al., “Long-term balancing selection drives evolution of immunity genes in Capsella,” eLife, vol. 8, Feb. 2019, doi: 10.7554/elife.43606.
<br>

[5]	J. L. Meitzler, S. Hinde, B. Bánfi, W. M. Nauseef, and P. R. Ortiz de Montellano, “Conserved Cysteine Residues Provide a Protein-Protein Interaction Surface in Dual Oxidase (DUOX) Proteins,” Journal of Biological Chemistry, vol. 288, no. 10, pp. 7147–7157, Mar. 2013, doi: 10.1074/jbc.m112.414797.
<br>

[6]	S. M. Marino and V. N. Gladyshev, “Cysteine Function Governs Its Conservation and Degeneration and Restricts Its Utilization on Protein Surfaces,” Journal of Molecular Biology, vol. 404, no. 5, pp. 902–916, Dec. 2010, doi: 10.1016/j.jmb.2010.09.027.
<br>

[7]	K. Kontos, P. Godard, B. André, J. van Helden, and G. Bontempi, “Machine learning techniques to identify putative genes involved in nitrogen catabolite repression in the yeast Saccharomyces cerevisiae,” BMC Proceedings, vol. 2, no. S4, Dec. 2008, doi: 10.1186/1753-6561-2-s4-s5.
<br>

[8]	X. Chen and H. Ishwaran, “Random forests for genomic data analysis,” Genomics, vol. 99, no. 6, pp. 323–329, Jun. 2012, doi: 10.1016/j.ygeno.2012.04.003.
<br>

[9]	W. G. Touw et al., “Data mining in the Life Sciences with Random Forest: a walk in the park or lost in the jungle?,” Briefings in Bioinformatics, vol. 14, no. 3, pp. 315–326, Jul. 2012, doi: 10.1093/bib/bbs034.
<br>

[10]	M. P. Slezina et al., “Molecular Insights into the Role of Cysteine-Rich Peptides in Induced Resistance to Fusarium oxysporum Infection in Tomato Based on Transcriptome Profiling,” International Journal of Molecular Sciences, vol. 22, no. 11, p. 5741, May 2021, doi: 10.3390/ijms22115741.
<br>

[11]	V. R. Joseph, “Optimal ratio for data splitting,” Statistical Analysis and Data Mining: The ASA Data Science Journal, vol. 15, no. 4, pp. 531–538, Apr. 2022, doi: 10.1002/sam.11583.
<br>

[12]	M. J. Feige, I. Braakman, and L. M. Hendershot, “CHAPTER 1.1. Disulfide Bonds in Protein Folding and Stability,” in Chemical Biology, Cambridge: Royal Society of Chemistry, 2018, pp. 1–33. Accessed: May 20, 2025. [Online]. Available: https://doi.org/10.1039/9781788013253-00001
<br>

[13]	M. Kusaba, K. Dwyer, J. Hendershot, J. Vrebalov, J. B. Nasrallah, and M. E. Nasrallah, “Self-Incompatibility in the Genus Arabidopsis: Characterization of the S Locus in the Outcrossing A. lyrata and Its Autogamous Relative A. thaliana,” The Plant Cell, vol. 13, no. 3, p. 627, Mar. 2001, doi: 10.2307/3871411.
<br>

[14]	C. Chu et al., “Strategies to Mitigate Age-Related Bias in Machine Learning: Scoping Review,” JMIR Aging, vol. 7, p. e53564, Mar. 2024, doi: 10.2196/53564.
<br>

[15]	J. Jumper and D. Hassabis, “Protein structure predictions to atomic accuracy with AlphaFold,” Nature Methods, vol. 19, no. 1, pp. 11–12, Jan. 2022, doi: 10.1038/s41592-021-01362-6.
<br>

[16]	A. Ceroni, A. Passerini, A. Vullo, and P. Frasconi, “DISULFIND: a disulfide bonding state and cysteine connectivity prediction server,” Nucleic Acids Research, vol. 34, no. Web Server, pp. W177–W181, Jul. 2006, doi: 10.1093/nar/gkl266.